# LoRA
## Low-Rank Adaptation
- LoRA冻结了预训练模型的权重
- 将可训练的秩分解矩阵注入到每个Transformer层


LoRA 线性层通过对线性层的预训练权重矩阵 $W_0 \in \mathbb{R}^{d \times k}$ 添加低秩分解来进行扩展。

$W_0 + \Delta W = W_0 + BA$

其中 $B \in \mathbb{R}^{d \times r}$，$A \in \mathbb{R}^{r \times k}$，且秩 $r \ll \min(d, k)$。

所有参数除 $A$ 和 $B$ 外都被冻结。

$\Delta W$ 在训练开始时初始化为零。

将 $x \Delta W^T$ 乘以 $\frac{\alpha}{r}$，其中 $\alpha$ 是一个超参数。一旦 $\alpha$ 被设定，就保持不变，仅调整 $r$。

In [1]:
import torch
import torch.nn as nn

In [5]:
class Linear(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool,
                 r: int, alpha: int = None):
        """
        :param in_features: 输入特征的数量
        :param out_features: 输出特征的数量
        :param bias: 标志位，指示是否有偏置参数
        :param r: 分解的秩 $r$
        :param alpha: 缩放因子 $\alpha$
        """
        super().__init__()
        # 将 $\frac{\alpha}{r} = 1$，即设置缩放因子 $\alpha = r$。
        if alpha is None:
            alpha = r
        # 预训练参数W0
        self.weight = nn.Parameter(torch.empty((out_features, in_features)))
        # 冻结参数
        self.weight.requires_grad = False
        if bias:
            # Bias $b_0$ (冻结)
            self.bias = nn.Parameter(torch.empty(out_features))
            self.bias.requires_grad = False
        else:
            self.bias = None
            
        self.scaling = alpha / r
        self.lora_a = nn.Parameter(torch.empty((r, in_features)))
        self.lora_b = nn.Parameter(torch.empty((out_features, r)))
        with torch.no_grad():
            # 1. 初始化 $A$，类似于普通线性层中的权重矩阵。 a=5 ** 0.5 用于在初始化时控制方差，使模型的梯度更稳定。
            # 2. 初始化 $B$ 为 $0$，以确保 $\Delta W = BA$ 在初始化时为 $0$。
            nn.init.kaiming_uniform_(self.lora_a, a=5 ** 0.5)
            nn.init.zeros_(self.lora_b)
    def forward(self, x: torch.Tensor):
        # 计算 $x W_0^T + b_0$
        result = nn.functional.linear(x, self.weight, bias=self.bias)
        result += (x @ self.lora_a.T @ self.lora_b.T) * self.scaling
        return result

## LoRA Embedding Layer

与 LoRA 线性层类似，此方法在预训练嵌入权重矩阵（$W_0 \in \mathbb{R}^{d \times k}$）中添加了一个低秩分解。

$W_0 + \Delta W = W_0 + BA$

In [4]:


class Embedding(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int,
                 r: int, alpha: int = None):
        super().__init__()

        if alpha is None:
            alpha = r

        self.weight = nn.Parameter(torch.empty((num_embeddings, embedding_dim)))
        self.weight.requires_grad = False

        self.scaling = alpha / r
        self.lora_a = nn.Parameter(torch.empty((r, num_embeddings)))
        self.lora_b = nn.Parameter(torch.empty((embedding_dim, r)))

        with torch.no_grad():
            nn.init.normal_(self.lora_a)
            nn.init.zeros_(self.lora_b)

    def forward(self, x: torch.Tensor):
        result = nn.functional.embedding(x, self.weight)
        result += (nn.functional.embedding(x, self.lora_a.T) @ self.lora_b.T) * self.scaling
        return result